In [2]:

import numpy as np
import pandas as pd
import matplotlib.pyplot
from google.cloud import bigquery #For BigQu
pd.options.display.max_colwidth = 100

In [3]:
#
def estimate_gigabytes_scanned(query, bq_client):
    # see https://cloud.google.com/bigquery/docs/reference/rest/v2/jobs#configuration.dryRun
    my_job_config = bigquery.job.QueryJobConfig()
    my_job_config.dry_run = True
    my_job = bq_client.query(query)

    return my_job

In [4]:
#Init
client = bigquery.Client()

hn_dataset_ref = client.dataset('nhtsa_traffic_fatalities', project='bigquery-public-data')
hn_dset = client.get_dataset(hn_dataset_ref)
hn_full = client.get_table(hn_dset.table('accident_2015'))

In [15]:
fifteen = 15
accidents_query_2015_all = """SELECT DISTINCT
                                acc15.consecutive_number,
                                month_of_crash,
                                 day_of_week,
                                 hour_of_crash,
                                 manner_of_collision_name,
                                 light_condition_name,
                                 land_use_name,
                                 latitude,
                                 longitude,
                                 atmospheric_conditions_1_name,
                                 driver_maneuvered_to_avoid_name,
                                 drivers_vision_obscured_by_name,
                                 number_of_drunk_drivers,
                                 sequence_of_events_name,
                                 area_of_Impact_associated_with_the_event_name,
                                 number_of_fatalities
                          FROM `bigquery-public-data.nhtsa_traffic_fatalities.accident_2015` AS acc15
                          JOIN `bigquery-public-data.nhtsa_traffic_fatalities.maneuver_2015` AS man15
                          ON acc15.consecutive_number = man15.consecutive_number
                          JOIN `bigquery-public-data.nhtsa_traffic_fatalities.vision_2015` AS vis15
                          ON acc15.consecutive_number = vis15.consecutive_number
                          JOIN `bigquery-public-data.nhtsa_traffic_fatalities.vsoe_2015` AS vseo15
                          ON acc15.consecutive_number = vseo15.consecutive_number
                          WHERE longitude < 0
                          AND longitude > -140
                      """ 

accidents_query_2016_all = """SELECT DISTINCT
                                 acc16.consecutive_number,
                                month_of_crash,
                                 day_of_week,
                                 hour_of_crash,
                                 manner_of_collision_name,
                                 light_condition_name,
                                 land_use_name,
                                 latitude,
                                 longitude,
                                 atmospheric_conditions_1_name,
                                 driver_maneuvered_to_avoid_name,
                                 drivers_vision_obscured_by_name,
                                 number_of_drunk_drivers,
                                 sequence_of_events_name,
                                 area_of_Impact_associated_with_the_event_name,
                                 number_of_fatalities
                          FROM `bigquery-public-data.nhtsa_traffic_fatalities.accident_2016` AS acc16
                          JOIN `bigquery-public-data.nhtsa_traffic_fatalities.maneuver_2016` AS man16
                          ON acc16.consecutive_number = man16.consecutive_number
                          JOIN `bigquery-public-data.nhtsa_traffic_fatalities.vision_2016` AS vis16
                          ON acc16.consecutive_number = vis16.consecutive_number
                          JOIN `bigquery-public-data.nhtsa_traffic_fatalities.vsoe_2016` AS vsoe16
                          ON acc16.consecutive_number = vsoe16.consecutive_number
                          WHERE longitude < 0
                          AND longitude > -140
                      """ 


In [16]:
accidents_2015 = estimate_gigabytes_scanned(accidents_query_2015_all, client)
accidents_2016 = estimate_gigabytes_scanned(accidents_query_2016_all, client)

In [17]:
df15 = accidents_2015.to_dataframe()
df16 = accidents_2016.to_dataframe()
#
frames = [df15, df16]
results = pd.concat(frames)
# results = df15


In [18]:
#clean up those pesky newline chars that have plaged me so.
results.replace('\\n',' ', regex=True, inplace=True)


In [19]:
#hundy = results.sample(n=100000)
results.to_csv("hightway_fatals_no.csv", index=False)
